In [54]:
from skimage import io,transform, util
import numpy as np
import stackview
import pystackreg

from os import path
from glob import glob
from pystackreg import StackReg
from skimage.transform import EuclideanTransform, warp

from imageUtils import find_most_likely_z_slice_using_CC, z_translate_and_pad
import pyvista as pv
pv.set_jupyter_backend('trame')

In [4]:
dirname = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Palbo senescence pilot/09-29-2025 Ear palbo pilot/M1tail 3017 H2B FUCCI/Left ear/1. Day 1/R1'

# 0. Load channel TIFs

In [5]:
B = io.imread(path.join(dirname,'B_reg.tif'))
R = io.imread(path.join(dirname,'R_reg.tif'))
R_shg = io.imread(path.join(dirname,'R_shg_reg.tif'))

dx = 0.2919921875
dz = 1
scale = [dz/dx,1,1]

In [9]:
slicer = stackview.slice(B,zoom_factor=0.5)
slicer

# Cross correlation 2.5-D method

In [16]:
idx_Z_reference = 12
idx_Z_target = find_most_likely_z_slice_using_CC(B[idx_Z_reference,...], R)

print(f'Z slice on target stsack: {idx_Z_target}')


Cross correlation started
Z slice on target stsack: 9


In [61]:
reference = B[idx_Z_reference]
target = R[idx_Z_target]
stackview.side_by_side(B[idx_Z_reference,...],R[idx_Z_target,...],zoom_factor=0.5)

side_by_side


In [64]:
#NB: Here, move the R channel wrt the B channel
print('StackReg + transform')
sr = StackReg(StackReg.RIGID_BODY)
T = sr.register(target/target.max(),reference) #Obtain the transformation matrices
T = EuclideanTransform(T)
R_transformed = np.zeros_like(R).astype(float)
R_shg_transformed = np.zeros_like(R).astype(float)
for i, R_slice in enumerate(R):
    R_transformed[i,...] = warp(R_slice,T)
    R_shg_transformed[i,...] = warp(R_shg[i,...],T)

# z-pad
print(f'Target image shape pre-padding: {R_transformed.shape}')
R_padded = z_translate_and_pad(B,R_transformed,idx_Z_reference,idx_Z_target)
print(f'Target image shape pre-padding: {R_padded.shape}')
print(f'Reference image shape: {B.shape}')

R_shg_padded = z_translate_and_pad(B,R_shg_transformed,idx_Z_reference,idx_Z_target)

R_padded = util.img_as_uint(R_padded/R_padded.max())
R_shg_padded = util.img_as_uint(R_shg_padded/R_shg_padded.max())

StackReg + transform
Target image shape pre-padding: (43, 1024, 1024)
Target image shape pre-padding: (52, 1024, 1024)
Reference image shape: (52, 1024, 1024)


In [63]:
stackview.side_by_side(B[idx_Z_reference,...],R_padded[idx_Z_reference,...],zoom_factor=0.5)


side_by_side


In [65]:
io.imsave(path.join(dirname,'R_reg_reg.tif'),R_padded)
io.imsave(path.join(dirname,'R_shg_reg_reg.tif'),R_shg_padded)

# BM mesh reconstruction and mesh-to-mesh registrastion

In [15]:
G_height_image = io.imread(path.join(dirname,'G_height_image.tif'))
bm_height_image = io.imread(path.join(dirname,'height_image.tif'))
stackview.slice(G_height_image,zoom_factor=0.5)

In [44]:
# Mesh reconstruction
from measurements import get_mesh_from_bm_image
bg_mesh = get_mesh_from_bm_image(bm_height_image,spacing=[1,1,1],decimation_factor=30)
G_mesh = get_mesh_from_bm_image(G_height_image,spacing=[1,1,1],decimation_factor=30)



/opt/miniconda3/envs/jupyter/lib/python3.12/site-packages/pyvista/core/utilities/points.py:79: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


In [45]:
pl = pv.Plotter()
pl.add_mesh(bg_mesh,color='red')
pl.add_mesh(G_mesh)
pl.show()

Widget(value='<iframe src="http://localhost:51403/index.html?ui=P_0x378e21df0_7&reconnect=auto" class="pyvista…

In [46]:
from trimesh import registration, transformations
from scipy.spatial.transform import Rotation

T,_ = registration.mesh_other(bg_mesh,G_mesh)

In [47]:
s = 1
r = Rotation.from_matrix(T[:3,:3]/s)
t = T[:3, 3]

# transforming mesh1
G_mesh_transformed = G_mesh.copy()
G_mesh_transformed.vertices = s * r.apply(G_mesh.vertices) + t[None, :]

pl = pv.Plotter()
pl.add_mesh(bg_mesh,color='red')
pl.add_mesh(G_mesh_transformed)
pl.show()

# saving mesh1 as STL file
# mesh1.export('rigidly_aligned_mesh.stl')

Widget(value='<iframe src="http://localhost:51403/index.html?ui=P_0x378e5e0c0_8&reconnect=auto" class="pyvista…

array([[ 9.99997545e-01, -1.33233687e-03,  1.77041980e-03,
        -4.64750274e-01],
       [ 1.33060772e-03,  9.99998637e-01,  9.77505090e-04,
         1.60286272e-01],
       [-1.77171976e-03, -9.75146956e-04,  9.99997955e-01,
         9.06021583e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])